In [1]:
from XQuant.Utils import *

## Config 常量

In [3]:
print(list(filter(lambda x:'__' not in x, Config.__dict__.keys())))

['database_dir', 'datasets_name', 'datatables', 'stock_table', 'stock_list', 'stock_num_list', 'futures_list', 'trade_date_table', 'trade_date_list', 'quarter_begin', 'quarter_end']


- database_dir: dict, 所有数据的文件夹路径
- datasets_name: list, database_dir的键
- datatables: dict, 具体的表名及相关信息
- stock_table: 股票信息表
- stock_list: 股票编码列表
- stock_num_list: 股票编码（仅数字部分）列表
- futures_list: 期货代号
- trade_date_table: 交易时间表
- trade_date_list: 可交易日期
- quarter_begin: 季度开始日期
- quarter_end: 季度结束日期

## TradeDate 时间管理类

In [2]:
# 判单是否属于日期类字符串/整形
print(is_date("2015-12-16"))
print(is_date("20151216"))
print(is_date(20151216))
print(is_date("2015年12月16日"))

True
True
True
True


In [3]:
# 日期标准化为pd.TimeStamp
format_date("2015年12月16日")

Timestamp('2015-12-16 00:00:00')

In [4]:
# 将区间[begin, end] 进行拓宽, 依据freq将拓展至指定位置
help(extend_date_span)

Help on function extend_date_span in module XQuant.Utils.Toolkit:

extend_date_span(begin: Union[str, int, datetime.datetime, datetime.date, pandas._libs.tslibs.timestamps.Timestamp], end: Union[str, int, datetime.datetime, datetime.date, pandas._libs.tslibs.timestamps.Timestamp], freq: Literal['Q', 'q', 'Y', 'y', 'M', 'm']) -> tuple[pandas._libs.tslibs.timestamps.Timestamp, pandas._libs.tslibs.timestamps.Timestamp]
    将区间[begin, end] 进行拓宽, 依据freq将拓展至指定位置, 详见下
    freq = M :
        [2018-01-04, 2018-04-20] -> [2018-01-01, 2018-04-30]
        [2018-01-01, 2018-04-20] -> [2018-01-01, 2018-04-30]
        [2018-01-04, 2018-04-30] -> [2018-01-01, 2018-04-30]
    freq = Q :
        [2018-01-04, 2018-04-20] -> [2018-01-01, 2018-06-30]
        [2018-01-01, 2018-04-20] -> [2018-01-01, 2018-06-30]
        [2018-01-04, 2018-06-30] -> [2018-01-01, 2018-06-30]
    freq = Y :
        [2018-01-04, 2018-04-20] -> [2018-01-01, 2018-12-31]
        [2018-01-01, 2018-04-20] -> [2018-01-01, 2018-12-31]
 

In [5]:
# 判断是否属于交易日
is_trade_date("20230811")

True

In [6]:
# 根据lag对交易日进行偏移
print(shift_trade_date("20230811", lag=1))
print(shift_trade_date("20230811", lag=-1))

2023-08-14 00:00:00
2023-08-10 00:00:00


In [2]:
# 返回两个日期之间的所有交易日的列表
print(range_trade_date('20230702', '20230709'))

[Timestamp('2023-07-03 00:00:00'), Timestamp('2023-07-04 00:00:00'), Timestamp('2023-07-05 00:00:00'), Timestamp('2023-07-06 00:00:00'), Timestamp('2023-07-07 00:00:00')]


In [3]:
# 根据偏移返回其间的交易日的列表
print(range_trade_date('20230702', lag=10))

[Timestamp('2023-07-03 00:00:00'), Timestamp('2023-07-04 00:00:00'), Timestamp('2023-07-05 00:00:00'), Timestamp('2023-07-06 00:00:00'), Timestamp('2023-07-07 00:00:00'), Timestamp('2023-07-10 00:00:00'), Timestamp('2023-07-11 00:00:00'), Timestamp('2023-07-12 00:00:00'), Timestamp('2023-07-13 00:00:00'), Timestamp('2023-07-14 00:00:00'), Timestamp('2023-07-17 00:00:00')]


## Formatter 格式化

is_date, date同TradeDate的is_date, format_date

In [10]:
# 股票期货代码格式化(仅支持单个)
print(format_stock('1'))
print(format_stock(1))
print(format_stock('0001'))
# Formatter.future

000001.SZ
000001.SZ
000001.SZ


In [11]:
# 股票期货代码格式化(支持向量)
format_code([1,2,3], kind='stock')

['000001.SZ', '000002.SZ', '000003.SZ']

In [12]:
# 对宽表类型的数据进行格式化
from XQuant.Collector import DataAPI

In [13]:
df = DataAPI.get_data(name='MktEqud', begin='20230101', fields=["ticker", "tradeDate", "marketValue"]).pivot(index='tradeDate', columns='ticker', values='marketValue')
df.head()

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,900941,900942,900943,900945,900946,900947,900948,900952,900953,900957
tradeDate,,,,,,,,,,,,,,,,,,,,,
2023-01-03,2.672195e+11,2.120278e+11,1.320509e+09,1.979464e+09,8.261969e+09,2.799300e+09,6.562677e+09,3.151799e+10,3.948922e+09,6.770323e+09,...,514960041.0,597361728.6,147987000.0,1.253253e+10,241621070.0,1.317088e+09,4.337591e+09,554634745.5,264320000.0,195091000.0
2023-01-04,2.778927e+11,2.217976e+11,1.323166e+09,1.979464e+09,8.315969e+09,2.726546e+09,6.785141e+09,3.133745e+10,3.959509e+09,6.919317e+09,...,517472041.2,598820487.4,145800000.0,1.257575e+10,240556660.0,1.317088e+09,4.272511e+09,550630162.5,262400000.0,195091000.0
2023-01-05,2.809977e+11,2.248216e+11,1.308553e+09,2.021805e+09,7.991970e+09,2.723081e+09,6.757333e+09,3.162116e+10,3.874814e+09,6.949116e+09,...,519984041.4,601008625.6,144585000.0,1.253253e+10,243749890.0,1.311820e+09,4.321321e+09,554634745.5,288640000.0,203816000.0
2023-01-06,2.837145e+11,2.235422e+11,1.276669e+09,2.000635e+09,7.748971e+09,2.709223e+09,6.701717e+09,3.167275e+10,4.107726e+09,6.812041e+09,...,518728041.3,585691658.2,143127000.0,1.231646e+10,241621070.0,1.306552e+09,4.448228e+09,544623288.0,296960000.0,203118000.0
2023-01-09,2.872076e+11,2.184247e+11,1.287297e+09,1.990049e+09,7.694972e+09,2.757726e+09,6.673909e+09,3.144062e+10,3.938335e+09,6.770323e+09,...,517472041.2,590797314.0,143613000.0,1.248932e+10,245878710.0,1.306552e+09,4.532832e+09,550630162.5,295040000.0,203467000.0


In [14]:
format_dataframe(df).head()

ticker,600000.SH,600001.SH,600002.SH,600003.SH,600004.SH,600005.SH,600006.SH,600007.SH,600008.SH,600009.SH,...,301387.SZ,301388.SZ,301389.SZ,301391.SZ,301396.SZ,301398.SZ,301408.SZ,301419.SZ,301429.SZ,301439.SZ
2023-01-03,2.122162e+11,NaN,NaN,NaN,3.445942e+10,NaN,1.132000e+10,1.624747e+10,2.114090e+10,1.399771e+11,...,NaN,2.476158e+09,5.406345e+09,6.194120e+09,3.317076e+09,2.526403e+09,NaN,NaN,NaN,NaN
2023-01-04,2.145644e+11,NaN,NaN,NaN,3.460142e+10,NaN,1.132000e+10,1.660002e+10,2.121431e+10,1.404250e+11,...,NaN,2.456693e+09,5.315625e+09,6.232880e+09,3.276869e+09,2.452003e+09,NaN,NaN,NaN,NaN
2023-01-05,2.157385e+11,NaN,NaN,NaN,3.552444e+10,NaN,1.132000e+10,1.656980e+10,2.121431e+10,1.451780e+11,...,NaN,2.482305e+09,5.491395e+09,6.177800e+09,3.290576e+09,2.538403e+09,NaN,NaN,NaN,NaN
2023-01-06,2.154450e+11,NaN,NaN,NaN,3.453042e+10,NaN,1.134000e+10,1.628776e+10,2.106750e+10,1.418434e+11,...,NaN,2.540700e+09,5.446035e+09,6.387240e+09,3.262248e+09,2.550403e+09,NaN,NaN,NaN,NaN
2023-01-09,2.154450e+11,NaN,NaN,NaN,3.609245e+10,NaN,1.144000e+10,1.617696e+10,2.114090e+10,1.456508e+11,...,NaN,2.564263e+09,5.451705e+09,6.738800e+09,3.279610e+09,2.680003e+09,NaN,NaN,NaN,NaN


## Tools 工具类

Tools.watcher

用作修饰器，监察函数运行时间

In [5]:
# 获取某个数据表的最新文件路径
get_newest_file('MktEqud')

'E:\\Share\\Stk_Data\\dataFile\\MktEqud\\MktEqud_Y2023_Q4.h5'

In [6]:
# 根据某个关键词搜索其对应数据表
help(search_keyword)

Help on function search_keyword in module XQuant.Utils.Toolkit:

search_keyword(keyword: str, fuzzy: bool = True, limit: int = 5, update: bool = False, initial_path: str = WindowsPath('E:/yuankangrui/dev_XQuant/XQuant/Temp/attrs.json'), **kwargs)
    :param initial_path: The initialization path of the log file
    :param keyword: the content you want to search for
    :param fuzzy: fuzzy matching or not
    :param limit: number of the results
    :param update: forced updating
    :return:



In [7]:
search_keyword("PE", fuzzy=True, limit=3, update=True)

缓存文件被保存至E:\yuankangrui\dev_XQuant\XQuant\Temp\attrs.json


正在初始化...: 100%|██████████| 447/447 [00:00<00:00, 538.97it/s, 状态=ResConSecCorederi_sheet 写入成功]                            


{'PE': ['MktEqudEval', 'FdmtIndiPSPit'],
 'conPe': ['ResConIndex',
  'ResConIndustryCitic',
  'ResConIndustrySw',
  'ResConSecCorederi',
  'ResConSecCorederi_sheet'],
 'conPeg1': ['ResConIndex',
  'ResConIndustryCitic',
  'ResConIndustrySw',
  'ResConSecCorederi']}

In [8]:
help(varname)

Help on function varname in module XQuant.Utils.Toolkit:

varname(p)
    将变量转化为变量名（字符串）
    :param p: 想要搜寻的变量
    :return: 变量的名字



In [10]:
varname(Config)

'Config'

In [11]:
# 检测主进程使用内存量
memory_analysis()

Memory Usage: 264.6M
